In [1]:
from bs4 import BeautifulSoup
import os,io
import parser,urllib.request,requests
from lxml import html,etree
import re
import pandas as pd

import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [9]:
opener = urllib.request.FancyURLopener({})
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
f = pd.read_html(url,header=0)
df = pd.DataFrame(f[0])
df.head()

C:\Users\sypar\Anaconda3\envs\tsa_course\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'},inplace=True)

Ignore cells of 'Not assigned'

In [11]:
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True,inplace=True)

In [12]:
coordinates = requests.get("http://cocl.us/Geospatial_data").content
coor = pd.read_csv(io.StringIO(coordinates.decode('utf-8')))
coor.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [13]:
df_2 = pd.merge(df,coor,left_on='PostalCode', right_on='PostalCode')
df_2.drop(['PostalCode'],axis=1,inplace=True)
df_2.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,North York,Lawrence Heights,43.718518,-79.464763
4,North York,Lawrence Manor,43.718518,-79.464763


In [14]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Latitude, Longitude))

# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_2['Latitude'], df_2['Longitude'], df_2['Borough'], df_2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Explore and cluster the neighborhoods

In [15]:
df.groupby('Borough').count()

,PostalCode,Neighborhood
Borough,,
Central Toronto,17,17
Downtown Toronto,37,37
East Toronto,7,7
East York,6,6
Etobicoke,44,44
Mississauga,1,1
North York,38,38
Queen's Park,1,1
Scarborough,37,37


In [16]:

# one hot encoding
toronto_onehot = pd.get_dummies(df[['Neighborhood']], prefix="", prefix_sep="")

#add encoded neighborhood to df
toronto_onehot['Borough'] = df['Borough'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Borough,Adelaide,Agincourt,Agincourt North,Albion Gardens,Alderwood,Bathurst Manor,Bathurst Quay,Bayview Village,Beaumond Heights,Bedford Park,Berczy Park,Birch Cliff,Bloordale Gardens,Brockton,Business Reply Mail Processing Centre 969 Eastern,CFB Toronto,CN Tower,Cabbagetown,Caledonia-Fairbanks,Canada Post Gateway Processing Centre,Cedarbrae,Central Bay Street,Chinatown,Christie,Church and Wellesley,Clairlea,Clarks Corners,Cliffcrest,Cliffside,Cliffside West,Cloverdale,Commerce Court,Davisville,Davisville North,Deer Park,Del Ray,Design Exchange,Don Mills North,Don Mills South,Dorset Park,Dovercourt Village,Downsview,Downsview Central,Downsview East,Downsview North,Downsview Northwest,Downsview West,Dufferin,East Birchmount Park,East Toronto,Emery,Eringate,Exhibition Place,Fairview,First Canadian Place,Flemingdon Park,Forest Hill North,Forest Hill SE,Forest Hill West,Garden District,Glencairn,Golden Mile,Grange Park,Guildwood,Harbord,Harbourfront,Harbourfront East,Harbourfront West,Henry Farm,High Park,Highland Creek,Hillcrest Village,Humber Bay,Humber Bay Shores,Humber Summit,Humbergate,Humberlea,Humewood-Cedarvale,India Bazaar,Ionview,Island airport,Islington,Jamestown,Keelesdale,Kennedy Park,Kensington Market,King,King and Spadina,King's Mill Park,Kingsview Village,Kingsway Park South East,Kingsway Park South West,L'Amoreaux East,L'Amoreaux West,Lawrence Heights,Lawrence Manor,Lawrence Manor East,Lawrence Park,Leaside,Little Portugal,Long Branch,Malvern,Markland Wood,Martin Grove,Martin Grove Gardens,Maryvale,Milliken,Mimico NE,Mimico NW,Mimico South,Montgomery Road,Moore Park,Morningside,Mount Dennis,Mount Olive,New Toronto,Newtonbrook,North Midtown,North Park,North Toronto West,Northwest,Northwood Park,Not assigned,Oakridge,Old Burnhamthorpe,Old Mill North,Old Mill South,Oriole,Parkdale,Parkdale Village,Parkview Hill,Parkwoods,Port Union,Princess Gardens,Queen's Park,Railway Lands,Rathnelly,Richmond,Richview Gardens,Riverdale,Roncesvalles,Rosedale,Roselawn,Rouge,Rouge Hill,Royal York South East,Royal York South West,Runnymede,Ryerson,Scarborough Town Centre,Scarborough Village,Scarborough Village West,Silver Hills,Silverstone,Silverthorn,South Hill,South Niagara,South Steeles,South of Bloor,St. James Town,St. Phillips,Steeles East,Stn A PO Boxes 25 The Esplanade,Studio District,Sullivan,Summerhill East,Summerhill West,Sunnylea,Swansea,Tam O'Shanter,The Annex,The Beaches,The Beaches West,The Danforth West,The Junction North,The Junction South,The Kingsway,The Queensway East,The Queensway West,Thistletown,Thorncliffe Park,Toronto Dominion Centre,Toronto Islands,Trinity,Underground city,Union Station,University of Toronto,Upper Rouge,Upwood Park,Victoria Hotel,Victoria Village,West Deane Park,West Hill,Westmount,Weston,Wexford,Wexford Heights,Willowdale,Willowdale South,Willowdale West,Wilson Heights,Woburn,Woodbine Gardens,Woodbine Heights,York Mills,York Mills West,York University,Yorkville
0,North York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [17]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped.head()

,Borough,Adelaide,Agincourt,Agincourt North,Albion Gardens,Alderwood,Bathurst Manor,Bathurst Quay,Bayview Village,Beaumond Heights,Bedford Park,Berczy Park,Birch Cliff,Bloordale Gardens,Brockton,Business Reply Mail Processing Centre 969 Eastern,CFB Toronto,CN Tower,Cabbagetown,Caledonia-Fairbanks,Canada Post Gateway Processing Centre,Cedarbrae,Central Bay Street,Chinatown,Christie,Church and Wellesley,Clairlea,Clarks Corners,Cliffcrest,Cliffside,Cliffside West,Cloverdale,Commerce Court,Davisville,Davisville North,Deer Park,Del Ray,Design Exchange,Don Mills North,Don Mills South,Dorset Park,Dovercourt Village,Downsview,Downsview Central,Downsview East,Downsview North,Downsview Northwest,Downsview West,Dufferin,East Birchmount Park,East Toronto,Emery,Eringate,Exhibition Place,Fairview,First Canadian Place,Flemingdon Park,Forest Hill North,Forest Hill SE,Forest Hill West,Garden District,Glencairn,Golden Mile,Grange Park,Guildwood,Harbord,Harbourfront,Harbourfront East,Harbourfront West,Henry Farm,High Park,Highland Creek,Hillcrest Village,Humber Bay,Humber Bay Shores,Humber Summit,Humbergate,Humberlea,Humewood-Cedarvale,India Bazaar,Ionview,Island airport,Islington,Jamestown,Keelesdale,Kennedy Park,Kensington Market,King,King and Spadina,King's Mill Park,Kingsview Village,Kingsway Park South East,Kingsway Park South West,L'Amoreaux East,L'Amoreaux West,Lawrence Heights,Lawrence Manor,Lawrence Manor East,Lawrence Park,Leaside,Little Portugal,Long Branch,Malvern,Markland Wood,Martin Grove,Martin Grove Gardens,Maryvale,Milliken,Mimico NE,Mimico NW,Mimico South,Montgomery Road,Moore Park,Morningside,Mount Dennis,Mount Olive,New Toronto,Newtonbrook,North Midtown,North Park,North Toronto West,Northwest,Northwood Park,Not assigned,Oakridge,Old Burnhamthorpe,Old Mill North,Old Mill South,Oriole,Parkdale,Parkdale Village,Parkview Hill,Parkwoods,Port Union,Princess Gardens,Queen's Park,Railway Lands,Rathnelly,Richmond,Richview Gardens,Riverdale,Roncesvalles,Rosedale,Roselawn,Rouge,Rouge Hill,Royal York South East,Royal York South West,Runnymede,Ryerson,Scarborough Town Centre,Scarborough Village,Scarborough Village West,Silver Hills,Silverstone,Silverthorn,South Hill,South Niagara,South Steeles,South of Bloor,St. James Town,St. Phillips,Steeles East,Stn A PO Boxes 25 The Esplanade,Studio District,Sullivan,Summerhill East,Summerhill West,Sunnylea,Swansea,Tam O'Shanter,The Annex,The Beaches,The Beaches West,The Danforth West,The Junction North,The Junction South,The Kingsway,The Queensway East,The Queensway West,Thistletown,Thorncliffe Park,Toronto Dominion Centre,Toronto Islands,Trinity,Underground city,Union Station,University of Toronto,Upper Rouge,Upwood Park,Victoria Hotel,Victoria Village,West Deane Park,West Hill,Westmount,Weston,Wexford,Wexford Heights,Willowdale,Willowdale South,Willowdale West,Wilson Heights,Woburn,Woodbine Gardens,Woodbine Heights,York Mills,York Mills West,York University,Yorkville
0,Central Toronto,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.058824,0.058824,0.058824,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.058824,0.058824,0.058824,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.058824,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.058824,0.0,0.0,0.000000,0.000000,0.0,0.058824,0.0,0.058824,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.0,0.000000,0.058824,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.058824,0.000000,

In [18]:

k = 11

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)


kmeans = KMeans(n_clusters=k, random_state=42).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([ 7,  9,  5,  4,  8,  2,  0,  1, 10,  3])

In [19]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped[["Borough","Cluster Labels"]]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = df_2.join(toronto_merged.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,North York,Parkwoods,43.753259,-79.329656,0
1,North York,Victoria Village,43.725882,-79.315572,0
2,Downtown Toronto,Harbourfront,43.654260,-79.360636,9
3,North York,Lawrence Heights,43.718518,-79.464763,0
4,North York,Lawrence Manor,43.718518,-79.464763,0


In [20]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
toronto_merged['Cluster Labels'].fillna(0, inplace=True)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
#     print(cluster)
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters